In [1]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import numpy as np
import pandas as pd
import gc
from tensorflow.keras import backend as K

# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/new-plant-diseases-dataset


In [3]:
# Dataset paths
# For Kaggle kernel:
train = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

# train = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train"
# valid = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\valid"

In [4]:
# Configuration
image_size = (224, 224)  
batch_size = 32          

print(f"Image size: {image_size}")
print(f"Batch size: {batch_size}")

Image size: (224, 224)
Batch size: 32


In [16]:
# Create datasets WITHOUT caching
def create_datasets():
    """
    Create training and validation datasets WITHOUT caching.
    Data will be loaded fresh from disk each epoch.
    Uses minimal memory but slower training.
    """
    # Training dataset
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        train,
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=True
    )

    # Validation dataset
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        valid,
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=False
    )

    # Save class names
    class_names = train_dataset.class_names
    num_classes = len(class_names)

    print(f"Number of classes: {num_classes}")

    total_batches = train_dataset.cardinality().numpy()

    train_dataset = train_dataset.shuffle(
        buffer_size=100,
        reshuffle_each_iteration=True
    )

    portion = 0.5
    train_dataset = train_dataset.take(int(total_batches * portion))

    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomFlip("vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomContrast(0.2),
    ])

    normalization = tf.keras.layers.Rescaling(1./255)

    train_dataset = train_dataset.map(
        lambda x, y: (data_augmentation(x, training=True), y),
        num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)

    val_dataset = val_dataset.map(
        lambda x, y: (normalization(x), y),
        num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)

    return train_dataset, val_dataset, class_names, num_classes

print("Creating datasets...")
train_dataset, val_dataset, class_names, num_classes = create_datasets()
print("\n Datasets created successfully!")

Creating datasets...
Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Number of classes: 38

✓ Datasets created successfully!


In [21]:
def create_model(base_model_class, model_name, num_classes):

    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=image_size + (3,)
    )

    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        jit_compile=True
    )

    return model, base_model
print("Model Created")

Model Created


In [22]:
def fine_tune_model(model, base_model, num_layers_to_unfreeze=20):
   
    base_model.trainable = True

    for layer in base_model.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

print('Fine Tune model Created')

Fine Tune model Created


In [ ]:
def get_callbacks(model_name):
    """
    Callbacks that monitor TRAINING metrics only (not validation).
    Since we're not validating during training, we monitor 'accuracy' and 'loss' instead.
    """
    callbacks = [
        EarlyStopping(
            monitor='accuracy',  
            patience=5,
            restore_best_weights=True,
            verbose=1,
            mode='max'
        ),
        ReduceLROnPlateau(
            monitor='loss',  
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1,
            mode='min'
        ),
        ModelCheckpoint(
            filepath=f'{model_name}_best.weights.h5',
            monitor='accuracy',  
            save_best_only=True,
            save_weights_only=True,
            verbose=1,
            mode='max'
        )
    ]
    return callbacks

print('Made Callbacks')

Made Callbacks


In [ ]:
def train_single_model(model_name, base_model_class):
    """
    Train a model WITHOUT validation during training.
    Validation is performed ONLY AFTER training is complete.
    """
    print(f"="*80)
    print(f"TRAINING {model_name}")
    print(f"="*80)
    
    # Special preprocessing for ResNet50
    if model_name == 'ResNet50':
        print(f"\nCreating datasets with ResNet50-specific preprocessing...")
        from tensorflow.keras.applications.resnet50 import preprocess_input
        
        train_ds = tf.keras.preprocessing.image_dataset_from_directory(
            train,
            seed=123,
            image_size=(224, 224),
            batch_size=batch_size,
            label_mode='categorical',
            shuffle=True
        )
        
        val_ds = tf.keras.preprocessing.image_dataset_from_directory(
            valid,
            seed=123,
            image_size=(224, 224),
            batch_size=batch_size,
            label_mode='categorical',
            shuffle=False
        )
        
        total_batches = train_ds.cardinality().numpy()
        train_ds = train_ds.shuffle(buffer_size=100, reshuffle_each_iteration=True)
        
        portion = 0.5
        train_ds = train_ds.take(int(total_batches * portion))
        
        print(f"Using {int(total_batches * portion)} batches for training")
        
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomFlip("vertical"),
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomZoom(0.2),
            tf.keras.layers.RandomContrast(0.2),
        ])
        
        train_ds = train_ds.map(
            lambda x, y: (preprocess_input(data_augmentation(x, training=True)), y),
            num_parallel_calls=tf.data.AUTOTUNE
        ).prefetch(tf.data.AUTOTUNE)
        
        val_ds = val_ds.map(
            lambda x, y: (preprocess_input(x), y),
            num_parallel_calls=tf.data.AUTOTUNE
        ).prefetch(tf.data.AUTOTUNE)
        
        train_dataset_model = train_ds
        val_dataset_model = val_ds
    else:
        train_dataset_model = train_dataset
        val_dataset_model = val_dataset

    print(f"\nCreating {model_name} model...")
    model, base_model = create_model(base_model_class, model_name, num_classes)
    print(f"✓ {model_name} model created")

    print(f"\n[STAGE 1] Initial training with frozen base model...")
    print("NOTE: No validation during training - validation will run after training completes")
    initial_epochs = 20

    history_stage1 = model.fit(
        train_dataset_model,
        # validation_data=val_dataset_model,  # ← REMOVED
        epochs=initial_epochs,
        callbacks=get_callbacks(f"{model_name}_stage1")
    )
    
    # Validate AFTER Stage 1 training
    print(f"\n[VALIDATION AFTER STAGE 1]")
    val_loss_s1, val_acc_s1 = model.evaluate(val_dataset_model, verbose=1)
    print(f"Stage 1 Validation - Loss: {val_loss_s1:.4f}, Accuracy: {val_acc_s1*100:.2f}%")

    print(f"\n[STAGE 2] Fine-tuning with unfrozen top layers...")
    print("NOTE: No validation during training - validation will run after training completes")
    model = fine_tune_model(model, base_model, num_layers_to_unfreeze=30)
    fine_tune_epochs = 20
    
    history_stage2 = model.fit(
        train_dataset_model,
        # validation_data=val_dataset_model,  # ← REMOVED
        epochs=fine_tune_epochs,
        callbacks=get_callbacks(f"{model_name}_stage2")
    )
    
    # Validate AFTER Stage 2 training
    print(f"\n[VALIDATION AFTER STAGE 2]")
    val_loss_s2, val_acc_s2 = model.evaluate(val_dataset_model, verbose=1)
    print(f"Stage 2 Validation - Loss: {val_loss_s2:.4f}, Accuracy: {val_acc_s2*100:.2f}%")

    model.save(f"{model_name}_plant_disease_model.h5")
    print(f"\n✓ {model_name} model saved")

    print(f"\nGenerating predictions for {model_name}...")
    y_pred = model.predict(val_dataset_model)

    y_true = []
    for image_batch, label_batch in val_dataset_model:
        y_true.append(label_batch)
    y_true = tf.concat(y_true, axis=0).numpy()

    np.save(f"{model_name}_y_pred.npy", y_pred)
    np.save(f"{model_name}_y_true.npy", y_true)
    print(f"✓ {model_name} predictions saved")

    y_pred_labels = y_pred.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)
    accuracy = np.mean(y_pred_labels == y_true_labels)

    print(f"\n{model_name} FINAL VALIDATION ACCURACY: {accuracy*100:.2f}%")

    if accuracy >= 0.985:
        print(f"✓ {model_name} achieved >=98.5% accuracy!")
    else:
        print(f"✗ {model_name} did not reach 98.5% accuracy (got {accuracy*100:.2f}%)")

    del model, base_model
    if model_name == 'ResNet50':
        del train_ds, val_ds, train_dataset_model, val_dataset_model
    K.clear_session()
    gc.collect()

    return accuracy

print("Fixed helper function defined")

✓ Fixed helper function defined


---
## Model Training Cells
**Run each cell below to train that specific model. Skip any cell to exclude that model from training.**

---

In [ ]:
# Train VGG16
try:
    vgg16_accuracy = train_single_model('VGG16', VGG16)
except Exception as e:
    print(f" Error training VGG16: {str(e)}")

In [ ]:
# Train VGG19
try:
    vgg19_accuracy = train_single_model('VGG19', VGG19)
except Exception as e:
    print(f" Error training VGG19: {str(e)}")


In [11]:
# Train InceptionV3
try:
    inceptionv3_accuracy = train_single_model('InceptionV3', InceptionV3)
except Exception as e:
    print(f" Error training InceptionV3: {str(e)}")



TRAINING InceptionV3

Creating InceptionV3 model...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
✓ InceptionV3 model created

[STAGE 1] Initial training with frozen base model...
Epoch 1/15
218/219 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.2197 - loss: 3.0633
Epoch 1: val_accuracy improved from -inf to 0.61205, saving model to InceptionV3_stage1_best.weights.h5
219/219 ━━━━━━━━━━━━━━━━━━━━ 88s 315ms/step - accuracy: 0.2209 - loss: 3.0571 - val_accuracy: 0.6121 - val_loss: 1.2570 - learning_rate: 0.0010
Epoch 2/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.5462 - loss: 1.5186
Epoch 2: val_accuracy improved from 0.61205 to 0.71056, saving model to InceptionV3_stage1_best.weights.h5
219/219 ━━━━━━━━━━━━━━━━━━━━ 60s 264ms/step - accuracy: 0.5463 - loss: 1.5182 - val_accuracy: 0.7106 - val_loss: 0.9119 - learning_rate: 0.0010
Epoch 3/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.6280 - loss: 1.2164
Epoch 3: val_accuracy improved from 0.71056 to 0.74


✓ InceptionV3 model saved

Generating predictions for InceptionV3...
550/550 ━━━━━━━━━━━━━━━━━━━━ 33s 49ms/step
✓ InceptionV3 predictions saved

InceptionV3 FINAL VALIDATION ACCURACY: 92.06%
✗ InceptionV3 did not reach 98.5% accuracy (got 92.06%)


In [ ]:
# Train Xception
try:
    xception_accuracy = train_single_model('Xception', Xception)
except Exception as e:
    print(f" Error training Xception: {str(e)}")


In [ ]:
try:
    resnet50_accuracy = train_single_model('ResNet50', ResNet50)
except Exception as e:
    print(f" Error training ResNet50: {str(e)}")


TRAINING ResNet50

Creating datasets with ResNet50-specific preprocessing...
Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Using 1098 batches for training

Creating ResNet50 model...
✓ ResNet50 model created

[STAGE 1] Initial training with frozen base model...
Epoch 1/20
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.6210 - loss: 1.3611
Epoch 1: val_accuracy improved from -inf to 0.91555, saving model to ResNet50_stage1_best.weights.h5
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 254s 216ms/step - accuracy: 0.6211 - loss: 1.3606 - val_accuracy: 0.9155 - val_loss: 0.2477 - learning_rate: 0.0010
Epoch 2/20
1097/1098 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.8687 - loss: 0.4122
Epoch 2: val_accuracy improved from 0.91555 to 0.92562, saving model to ResNet50_stage1_best.weights.h5
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 228s 206ms/step - accuracy: 0.8687 - loss: 0.4122 - val_accuracy: 0.9256 - val_loss: 0.2117 - learning_rate: 0.0010
Epoch 3/20
1098/109

In [ ]:
# Train DenseNet121
try:
    densenet121_accuracy = train_single_model('DenseNet121', DenseNet121)
except Exception as e:
    print(f" Error training DenseNet121: {str(e)}")


---
## Results & Comparison
**Run the cells below after training your models to see the comparison.**

---

In [ ]:
# Generate comparison table
from sklearn.metrics import precision_score, recall_score, f1_score

print("="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)

# List of all possible models
all_models = ['VGG16', 'VGG19', 'InceptionV3', 'Xception', 'ResNet50', 'DenseNet121']
results = []

for model_name in all_models:
    try:
        y_pred = np.load(f"{model_name}_y_pred.npy")
        y_true = np.load(f"{model_name}_y_true.npy")

        y_pred_labels = y_pred.argmax(axis=1)
        y_true_labels = y_true.argmax(axis=1)

        accuracy = np.mean(y_pred_labels == y_true_labels)
        precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
        recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
        f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')

        top5_pred = np.argsort(y_pred, axis=1)[:, -5:]
        top5_accuracy = np.mean([y_true_labels[i] in top5_pred[i] for i in range(len(y_true_labels))])

        results.append({
            'Model': model_name,
            'Accuracy (%)': accuracy * 100,
            'Precision (%)': precision * 100,
            'Recall (%)': recall * 100,
            'F1-Score (%)': f1 * 100,
            'Top-5 Accuracy (%)': top5_accuracy * 100,
            'Meets Target (>98.5%)': '✓' if accuracy > 0.985 else '✗'
        })
    except FileNotFoundError:
        print(f"{model_name} was not trained (skipped)")
    except Exception as e:
        print(f"Could not load results for {model_name}: {str(e)}")

if len(results) > 0:
    comparison_df = pd.DataFrame(results)
    comparison_df = comparison_df.sort_values('Accuracy (%)', ascending=False)

    print("\n" + comparison_df.to_string(index=False))
    print("\n" + "="*80)

    models_above_target = comparison_df[comparison_df['Meets Target (>98.5%)'] == '✓'].shape[0]
    print(f"\nModels achieving >98.5% accuracy: {models_above_target}/{len(results)}")

    comparison_df.to_csv('model_comparison_sequential.csv', index=False)
    print("\n Comparison table saved to 'model_comparison_sequential.csv'")
else:
    print("\n  No models were trained. Please run at least one model training cell above.")

In [ ]:
# Visualize comparison
import matplotlib.pyplot as plt
import seaborn as sns

if len(results) > 0:
    sns.set_style('whitegrid')
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    colors = ['green' if x == '✓' else 'red' for x in comparison_df['Meets Target (>98.5%)']]
    plt.barh(comparison_df['Model'], comparison_df['Accuracy (%)'], color=colors, alpha=0.7)
    plt.axvline(x=98.5, color='blue', linestyle='--', label='Target (98.5%)')
    plt.xlabel('Accuracy (%)')
    plt.title('Model Accuracy Comparison')
    plt.legend()
    plt.tight_layout()

    plt.subplot(1, 2, 2)
    metrics_df = comparison_df[['Model', 'Precision (%)', 'Recall (%)', 'F1-Score (%)']].set_index('Model')
    metrics_df.plot(kind='bar', ax=plt.gca(), width=0.8)
    plt.ylabel('Score (%)')
    plt.title('Model Metrics Comparison')
    plt.xticks(rotation=45, ha='right')
    plt.legend(loc='lower right')
    plt.tight_layout()

    plt.savefig('model_comparison_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(" Visualization saved to 'model_comparison_visualization.png'")
else:
    print("  No results to visualize. Train at least one model first.")

In [ ]:
# Generate detailed classification reports
from sklearn.metrics import classification_report

print("DETAILED CLASSIFICATION REPORTS")

for model_name in all_models:
    try:
        y_pred = np.load(f"{model_name}_y_pred.npy")
        y_true = np.load(f"{model_name}_y_true.npy")

        y_pred_labels = y_pred.argmax(axis=1)
        y_true_labels = y_true.argmax(axis=1)

        print(f"{model_name} - Classification Report")
       

        report = classification_report(y_true_labels, y_pred_labels, target_names=class_names)
        print(report)

        # Save report
        with open(f"{model_name}_classification_report.txt", 'w') as f:
            f.write(f"{model_name} - Classification Report\n")
            f.write("="*80 + "\n")
            f.write(report)

        print(f" Report saved to '{model_name}_classification_report.txt'")
    except FileNotFoundError:
        print(f"\n  {model_name} was not trained (skipped)")
    except Exception as e:
        print(f" Could not generate report for {model_name}: {str(e)}")

In [ ]:
# Final summary
print("FINAL SUMMARY")


if len(results) > 0:
    print(f"\nTotal models trained: {len(results)}")
    print(f"Models achieving >98.5% accuracy: {models_above_target}")
    print(f"\nBest performing model: {comparison_df.iloc[0]['Model']}")
    print(f"Best accuracy: {comparison_df.iloc[0]['Accuracy (%)']:.2f}%")
else:
    print("\n  No models were trained.")

print("\nKey features implemented:")
print("   Sequential training (one model at a time)")
print("   Separate cells for each model (skip any model)")
print("   NO caching (minimal memory usage)")
print("   Memory management between models")
print("   GPU memory growth enabled")
print("   Mixed precision training")
print("   Two-stage training with fine-tuning")
print("   Enhanced data augmentation")
print("   Learning rate scheduling")
print("   Early stopping and model checkpointing")
print("\n" + "="*80)
print("TRAINING COMPLETE!")
